# RTP Update

### Setup

In [176]:
# import packages
import pandas as pd
import pathlib
from pathlib import Path
import os
import arcpy
from utils import *
import numpy as np
import pickle
# external connection packages
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

# pandas options
pd.options.mode.copy_on_write = True
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
pd.options.display.max_rows    = 999

# my workspace 
workspace = r"C:\Users\mbindl\Desktop\Workspace.gdb"
# current working directory
local_path = pathlib.Path().absolute()
# set data path as a subfolder of the current working directory TravelDemandModel\2022\
data_dir = local_path.parents[0] / 'data'
# folder to save processed data
out_dir  = local_path.parents[0] / 'data/processed_data'
# workspace gdb for stuff that doesnt work in memory
# gdb = os.path.join(local_path,'Workspace.gdb')
gdb = workspace
# set environement workspace to in memory 
arcpy.env.workspace = 'memory'
# # clear memory workspace
# arcpy.management.Delete('memory')

# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)

# get parcels from the database
# network path to connection files
filePath = "F:/GIS/PARCELUPDATE/Workspace/"
# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")
sdeEdit    = os.path.join(filePath, "Edit.sde")

# Pickle variables
# part 1 - spatial joins and new categorical fields
parcel_pickle_part1    = data_dir / 'parcel_pickle1.pkl'
# part 2 - forecasting applied
parcel_pickle_part2    = data_dir / 'parcel_pickle2.pkl'

# columsn to list
initial_columns = [ 'APN',
                    'APO_ADDRESS',
                    'Residential_Units',
                    'TouristAccommodation_Units',
                    'CommercialFloorArea_SqFt',
                    'YEAR',
                    'JURISDICTION',
                    'COUNTY',
                    'OWNERSHIP_TYPE',
                    'COUNTY_LANDUSE_DESCRIPTION',
                    'EXISTING_LANDUSE',
                    'REGIONAL_LANDUSE',
                    'YEAR_BUILT',
                    'PLAN_ID',
                    'PLAN_NAME',
                    'ZONING_ID',
                    'ZONING_DESCRIPTION',
                    'TOWN_CENTER',
                    'LOCATION_TO_TOWNCENTER',
                    'TAZ',
                    'PARCEL_ACRES',
                    'PARCEL_SQFT',
                    'WITHIN_BONUSUNIT_BNDY',
                    'WITHIN_TRPA_BNDY',
                    'SHAPE']

# schema for the final output
final_schema = [ 'APN',
                'APO_ADDRESS',
                'Residential_Units',
                'TouristAccommodation_Units',
                'CommercialFloorArea_SqFt',
                'YEAR',
                'JURISDICTION',
                'COUNTY',
                'OWNERSHIP_TYPE',
                'COUNTY_LANDUSE_DESCRIPTION',
                'EXISTING_LANDUSE',
                'REGIONAL_LANDUSE',
                'YEAR_BUILT',
                'PLAN_ID',
                'PLAN_NAME',
                'ZONING_ID',
                'ZONING_DESCRIPTION',
                'TOWN_CENTER',
                'LOCATION_TO_TOWNCENTER',
                'TAZ',
                'PARCEL_ACRES',
                'PARCEL_SQFT',
                'WITHIN_BONUSUNIT_BNDY',
                'WITHIN_TRPA_BNDY',
                'IPES_SCORE',
                'IPES_SCORE_TYPE',
                'RETIRED',
                'HOUSING_ZONING',
                'MAX_RESIDENTIAL_UNITS', 
                'MAX_COMMERCIAL_FLOOR_AREA', 
                'MAX_TAU_UNITS',
                'SHAPE']

In [177]:
def get_conn(db):
    # Get database user and password from environment variables on machine running script
    db_user             = os.environ.get('DB_USER')
    db_password         = os.environ.get('DB_PASSWORD')
    # driver is the ODBC driver for SQL Server
    driver              = 'ODBC Driver 17 for SQL Server'
    # server names are
    sql_12              = 'sql12'
    sql_14              = 'sql14'
    # make it case insensitive
    db = db.lower()
    # make sql database connection with pyodbc
    if db   == 'sde_tabular':
        connection_string = f"DRIVER={driver};SERVER={sql_12};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    elif db == 'tahoebmpsde':
        connection_string = f"DRIVER={driver};SERVER={sql_14};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    elif db == 'sde':
        connection_string = f"DRIVER={driver};SERVER={sql_12};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    # else return None
    else:
        engine = None
    # connection file to use in pd.read_sql
    return engine

# save to pickle
def to_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f'{filename} pickled')

# save to pickle and feature class
def to_pickle_fc(data, filename):
    data.spatial.to_featureclass(filename)
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f'{filename} pickled and saved as feature class')

# get a pickled file as a dataframe
def from_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    print(f'{filename} unpickled')
    return data

# function to get where Zoningin_ID Use_Type = Multi-Family and Density
def get_mf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Multiple Family Dwelling
    df = df.loc[df['Use_Type'] == 'Multiple Family Dwelling']
    return df[columns_to_keep]

def get_sf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Single Family Dwelling
    df = df.loc[df['Use_Type'] == 'Single Family Dwelling']
    return df[columns_to_keep]

def get_sf_only_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Single Family Dwelling and not Multiple Family Dwelling
    dfMF = get_mf_zones(df)
    dfSF = get_sf_zones(df)
    df = dfSF.loc[~dfSF['Zoning_ID'].isin(dfMF['Zoning_ID'])]
    return df[columns_to_keep]

def get_mf_only_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Multiple Family Dwelling and not Single Family Dwelling
    dfSF = get_sf_zones(df)
    dfMF = get_mf_zones(df)
    df = dfMF.loc[~dfMF['Zoning_ID'].isin(dfSF['Zoning_ID'])]
    return df[columns_to_keep]

def get_recieving_zones(df):
    columns_to_keep = ['Zoning_ID', 'SPECIAL_DESIGNATION']
    # filter transfer recieving
    df = df.loc[df['SPECIAL_DESIGNATION'] == 'Receive']
    return df[columns_to_keep]

def get_sending_zones(df):
    columns_to_keep = ['Zoning_ID', 'SPECIAL_DESIGNATION']
    df = df.loc[df['SPECIAL_DESIGNATION'] == 'Transfer']
    return df[columns_to_keep]


### Get Data

In [ ]:
# parcel development layer polygons
parcel_db = Path(sdeEdit) / "SDE.Parcel\\SDE.Parcel_History_Attributed"
# query 2022 rows
sdf_units = pd.DataFrame.spatial.from_featureclass(parcel_db)
sdf_units = sdf_units.loc[sdf_units['YEAR'] == 2022]
sdf_units.spatial.sr = sr

# # get parcel level data from Collection SDE
# vhr feature layer polygons 
vhr_db = Path(sdeCollect) / "SDE.Parcel\\SDE.Parcel_VHR"
sdf_vhr = pd.DataFrame.spatial.from_featureclass(vhr_db)
sdf_vhr.spatial.sr = sr
# filter vhr layer to active status
sdf_vhr = sdf_vhr.loc[sdf_vhr['Status'] == 'Active']

# TAZ feature layer polygons
taz_db = Path(sdeBase) / "SDE.Transportation\\SDE.Transportation_Analysis_Zone"
# get as spatial dataframe
sdf_taz = pd.DataFrame.spatial.from_featureclass(taz_db)
# set spatial reference to NAD 1983 UTM Zone 10N
sdf_taz.spatial.sr = sr

# censuse feature class
census_fc    = Path(sdeBase) / "SDE.Census\\SDE.Tahoe_Census_Geography"
# bouns unit boundary feature class
bonus_unit_fc = Path(sdeBase) / "SDE.Planning\SDE.Bonus_unit_boundary"

# disable Z values on block group feature layer
with arcpy.EnvManager(outputZFlag="Disabled"):    
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features="F:\GIS\DB_CONNECT\Vector.sde\SDE.Census\SDE.Tahoe_Census_Geography",
        Output_Geodatabase=r"C:\Users\mbindl\Desktop\Workspace.gdb"
    )
# disable Z values on block group feature layer
with arcpy.EnvManager(outputZFlag="Disabled"):    
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features="F:\GIS\DB_CONNECT\Vector.sde\SDE.Planning\SDE.Bonus_unit_boundary",
        Output_Geodatabase=r"C:\Users\mbindl\Desktop\Workspace.gdb"
    )

# block group feature layer polygons with no Z
sdf_block = pd.DataFrame.spatial.from_featureclass(Path(gdb) / 'Tahoe_Census_Geography')
sdf_block = sdf_block.loc[(sdf_block['YEAR'] == 2020) & (sdf_block['GEOGRAPHY'] == 'Block Group')]
sdf_block.spatial.sr = sr

# bonus unit boundary wihtout Z
sdf_bonus = pd.DataFrame.spatial.from_featureclass(Path(gdb) / 'Bonus_unit_boundary')
sdf_bonus.spatial.sr = sr

# get parcel level data from LTinfo
dfIPES       = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfLCV_LTinfo = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetParcelsByLandCapability/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfRetired    = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfBankedDev  = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfTransacted = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfAllParcels = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')

# get use tables 
# zoning data
sde_engine = get_conn('sde')
with sde_engine.begin() as conn:
    df_uses    = pd.read_sql("SELECT * FROM sde.SDE.PermissibleUses", conn)
    df_special = pd.read_sql("SELECT * FROM sde.SDE.Special_Designation", conn)

### Parcel Data Engineering

In [ ]:
# spatial join to get TAZ
arcpy.SpatialJoin_analysis(sdf_units, sdf_taz, "Existing_Development_TAZ", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get Block Group
arcpy.SpatialJoin_analysis(sdf_units, sdf_block, "Existing_Development_BlockGroup", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join of Bonus Unit Boundary
arcpy.SpatialJoin_analysis(sdf_units, sdf_bonus, "Existing_Development_BonusUnitBoundary",
                            "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "INTERSECT")

In [ ]:
# spatial dataframe with only initial columns
sdfParcels = sdf_units[initial_columns]

# get results of spatial joins as spatial dataframes
sdf_units_taz   = pd.DataFrame.spatial.from_featureclass("Existing_Development_TAZ", sr=sr)  
sdf_units_block = pd.DataFrame.spatial.from_featureclass("Existing_Development_BlockGroup", sr=sr)
sdf_units_bonus = pd.DataFrame.spatial.from_featureclass("Existing_Development_BonusUnitBoundary", sr=sr)
# cast to string
sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'] = sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'].astype(str)
sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'] = 'No'
# if Id is not NA then within bonus unit boundary = yes, else
sdf_units_bonus.loc[sdf_units_bonus['Id'].notna(), 'WITHIN_BONUSUNIT_BNDY'] = 'Yes'

# map dictionary to sdf_units dataframe to fill in TAZ and Block Group fields
sdfParcels['TAZ']                   = sdfParcels.APN.map(dict(zip(sdf_units_taz.APN,   sdf_units_taz.TAZ_1)))
sdfParcels['BLOCK_GROUP']           = sdfParcels.APN.map(dict(zip(sdf_units_block.APN, sdf_units_block.TRPAID)))
# map IPES score to parcels
sdfParcels['IPES_SCORE']            = sdfParcels['APN'].map(dict(zip(dfIPES.APN, dfIPES.IPESScore)))
sdfParcels['IPES_SCORE_TYPE']       = sdfParcels['APN'].map(dict(zip(dfIPES.APN, dfIPES.IPESScoreType)))
# retired parcels
sdfParcels['RETIRED']               = sdfParcels['APN'].map(dict(zip(dfAllParcels.APN, dfAllParcels.RetiredFromDevelopment)))
sdfParcels['WITHIN_BONUSUNIT_BNDY'] = sdfParcels['APN'].map(dict(zip(sdf_units_bonus.APN, sdf_units_bonus.WITHIN_BONUSUNIT_BNDY)))
# define housnig zoning and density
sdfParcels['HOUSING_ZONING']        = 'NA'

# if the zoning id is in the list of single family zones then set to SF
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_sf_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'SF'
# if the zoning id is in the list of multiple family zones then set to MF
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_mf_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'MF'
# if the zoning id is in the list of single family zones and not in the multiple family zones then set to SF only
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_sf_only_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'SF_only'
# if the zoning id is in the list of multiple family zones and not in the single family zones then set to MF only
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_mf_only_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'MF_only'

# get density for MF and MF only zones, max residential units, and adjusted residential units
dfMF = get_mf_zones(df_uses)
sdfParcels['DENSITY']                    = sdfParcels['ZONING_ID'].map(dict(zip(dfMF.Zoning_ID, dfMF.Density)))
sdfParcels['MAX_RESIDENTIAL_UNITS']      = sdfParcels['PARCEL_ACRES'] * sdfParcels['DENSITY']
sdfParcels['MAX_BUILDABLE_UNITS'] = sdfParcels['MAX_RESIDENTIAL_UNITS']*0.6

# if COUNTY is in EL or PL and SF allowed then set ADU_ALLOWED to yes
sdfParcels['ADU_ALLOWED'] = 'No'
sdfParcels.loc[(sdfParcels['COUNTY'].isin(['EL','PL'])) & (~sdfParcels['HOUSING_ZONING'].isin(['MF_only', 'NA'])), 'ADU_ALLOWED'] = 'Yes'
sdfParcels.loc[(sdfParcels['COUNTY'].isin(['WA','DG','CC'])) & (sdfParcels['PARCEL_ACRES']>=1) &(~sdfParcels['HOUSING_ZONING'].isin(['MF_only', 'NA'])), 'ADU_ALLOWED'] = 'Yes'

sdfParcels['POTENTIAL_BUILDABLE_UNITS'] = 0
sdfParcels.loc[(sdfParcels['Residential_Units']>0)&(sdfParcels['HOUSING_ZONING'].isin(['MF','MF_only','SF_only'])), 'POTENTIAL_BUILDABLE_UNITS'] = sdfParcels['MAX_BUILDABLE_UNITS']-sdfParcels['Residential_Units']

# export to pickle
sdfParcels.to_pickle(parcel_pickle_part1)
# to feature class
sdfParcels.spatial.to_featureclass(Path(gdb)/'Parcel_Base_2022')

In [ ]:
sdfParcels.info()

## Model Year Forecasting

### Residential Forecasting

1) Start with Known projects	
* Add Known Residential Allocations from 2023-2024
* Add Known Residential Bonus Units from permitted projects, applications in review, and RBU reservations in LT Info
* Add Known Accessory Dwelling permits not completed
* Remove Known Banking projects that removed units in 2023-2024
    
2) Assign Remaining Residential Bonus Units within Residential Bonus Unit Boundary	
* Identify	Vacant buildable lots within Bonus Unit boundary - calculate allowed density
* Assign remaining jurisdiction pool units to available parcels within jurisidiction
* Assign remaining TRPA pool units to available parcels throughout region (use adjusted weighting from existing residential units?)
* Calcuate Remaining local jurisdiction Residential Allocation pool units less units used for known projects
* Calcuate Remaining banked units less units used for known projects
* Calcuate Remaining converted units less units used for known projects
    
3) Evaluate Vacant Buildable Lots with Multi-family Residential Allowed Use	
* Identify	Vacant buildable lots with allowed Multi-family use, calculate allowed density
* Assign 15% of remaining local jurisdiction Residential Allocation pool units to available multi-family parcels within jurisidiction
* Assign 35% of remaining Banked units to available multi-family parcels (use adjusted weighting from existing residential units?)
* Assign 35% of remaining Converted units to available multi-family parcels (use adjusted weighting from existing residential units?)
    
4) Evaluate Vacant Buildable Lots with Single-family Residential Allowed Use	
* Identify	Vacant buildable lots with allowed Single-family use
* Assign 70% of local jurisdiction Residential Allocation pool units to available single-family parcels within jurisidiction (if enough vacant parcels are available, otherwise build all available parcels)
* Assign 50% of Banked units to available single-family parcels (use adjusted weighting from existing residential units?) (if enough vacant parcels are available, otherwise build all available parcels)
* Assign 50% of Converted units to available single-family parcels (use adjusted weighting from existing residential units?) (if enough vacant parcels are available)
    
5) Evaluate Underbuilt parcels with Multi-family Residential Allowed Use	
* Identify	Underbuilt Residential lots with allowed Multi-family use
* Assign 12% of local jurisdiction Residential Allocation pool units to underbuilt Multi-family parcels within jurisidiction
* Assign 12% of banked residential units to underbuilt Multi-family parcels (use adjusted weighting from existing residential units?)
* Assign 12% of converted residential units to underbuilt Multi-family parcels (use adjusted weighting from existing residential units?)
    
6) Evaluate Underbuilt parcels with Accessory Dwelling Uses Allowed (All California Parcels and NV Parcels Greater than 1 Acre)	
* Identify	Accessory Dwelling Uses Allowed (All California Parcels and NV Parcels Greater than 1 Acre)
* Assign remainder of local jurisdiction Residential Allocation pool units to ADUs
* Assign remainder of banked units to ADUs (weighted higher in CA due to limitations)
* Assign remainder of converted units to ADUs (weighted higher in CA due to limitations)


In [ ]:
# get pickle
sdfParcels = from_pickle(parcel_pickle_part1)
sdfParcels['FORECASTED_RESIDENTIAL_UNITS'] = 0
# fill na 0 and cast ADJUSTED_RESIDENTIAL_UNITS to int 
sdfParcels['MAX_BUILDABLE_UNITS'] = sdfParcels['MAX_BUILDABLE_UNITS'].fillna(0).astype(int)
sdfParcels.loc[sdfParcels['HOUSING_ZONING'] == 'SF_only', 'MAX_BUILDABLE_UNITS'] = 1
# set FORECAST_REASON to a null string
sdfParcels['FORECAST_REASON'] = ''
# cast as type str
sdfParcels['FORECAST_REASON'] = sdfParcels['FORECAST_REASON'].astype(str)

In [ ]:
# Randomly sort parcels so that development can be assigned randomly
sdfParcels = sdfParcels.sample(frac=1).reset_index(drop=True)
# Export index and apn to a csv file for future reference with a name that includes the date and time
# This will allow us to recreate the same random order in the future
sdfParcels[['APN']].to_csv(out_dir / f"Parcel_Sort_Order_{pd.Timestamp.now().strftime('%Y%m%d%H%M%S')}.csv", index=True)

In [183]:
def forecast_residential_units(df, condition, target_sum, reason):
    # filter to parcels available for development
    sdfAvailable = df.loc[eval(condition)]
    running_sum = 0
    rows_to_fill = []
    # Loop through the rows and fill the 'new_column'
    for idx, row in sdfAvailable.iterrows():
        # Calculate the remaining amount that can be filled
        remaining_amount = target_sum - running_sum
        if row['MAX_BUILDABLE_UNITS'] <= remaining_amount:
            # If the current row's value fits, add it to the column
            df.loc[idx, 'FORECASTED_RESIDENTIAL_UNITS'] = row['MAX_BUILDABLE_UNITS']
            running_sum += row['MAX_BUILDABLE_UNITS']
            rows_to_fill.append(idx)
        elif remaining_amount > 0:
            # If it exceeds the remaining amount, fill with the remaining value
            df.loc[idx, 'FORECASTED_RESIDENTIAL_UNITS'] = remaining_amount
            running_sum += remaining_amount
            rows_to_fill.append(idx)
            break
        else:
            break
    # reason for development
    df.loc[rows_to_fill, 'FORECAST_REASON'] = reason
    return df  

In [184]:
## Conditional Statements for Forecasting Residential Unit Development ##
# Within TRPA Boundary as condition for all
# Parcel criteria for development of CSLT Bonus Units
Base_Criteria = "(df['FORECAST_REASON'] == '') &"
CSLT_Bonus_condition   = "(df['FORECAST_REASON'] == '') & (df['JURISDICTION'] == 'CSLT') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes') & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['IPES_SCORE'] > 0) & (df['RETIRED'] == 'No')"
# Parcel criteria for development of CLST General Unit Pool
CSLT_General_Condition = "(df['FORECAST_REASON'] == '') & (df['JURISDICTION'] == 'CSLT') & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['IPES_SCORE'] > 0) & (df['RETIRED'] == 'No')"
# Parcel criteria for development of EL General Unit Pool
EL_General_Condition   = "(df['FORECAST_REASON'] == '') & (df['JURISDICTION'] == 'EL') & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['IPES_SCORE'] > 0) & (df['RETIRED'] == 'No')"
# Parcel criteria for development of PL General Unit Pool
PL_General_Condition   = "(df['FORECAST_REASON'] == '') & (df['JURISDICTION'] == 'PL') & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['IPES_SCORE'] > 726) & (df['RETIRED'] == 'No')"
# Parcel criteria for development of PL Bonus Units
PL_Bonus_Condition     = "(df['FORECAST_REASON'] == '') & (df['JURISDICTION'] == 'PL') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes') & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['IPES_SCORE'] > 726) & (df['RETIRED'] == 'No')"
# Parcel criteria for development of WA General Unit Pool
WA_General_Condition   = "(df['FORECAST_REASON'] == '') & (df['JURISDICTION'] == 'WA') & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['IPES_SCORE'] > 0) & (df['RETIRED'] == 'No')"
# Parcel criteria for development of WA Bonus Units
WA_Bonus_Condition     = "(df['FORECAST_REASON'] == '') & (df['JURISDICTION'] == 'WA') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes') & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['IPES_SCORE'] > 0) & (df['RETIRED'] == 'No')"
# Parcel criteria for development of DG General Unit Pool
DG_General_Condition   = "(df['FORECAST_REASON'] == '') & (df['JURISDICTION'] == 'DG') & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['IPES_SCORE'] > 0) & (df['RETIRED'] == 'No')"
# Parcel criteria for development of DG Bonus Units
DG_Bonus_Condition     = "(df['FORECAST_REASON'] == '') & (df['JURISDICTION'] == 'DG') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes') & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['IPES_SCORE'] > 0) & (df['RETIRED'] == 'No')"
# Parcel criteria for development of TRPA General Unit Pool
TRPA_General_Condition = "(df['FORECAST_REASON'] == '') & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['IPES_SCORE'] > 0) & (df['RETIRED'] == 'No')"
# Parcel criteria for development of TRPA Bonus Units
TRPA_Bonus_Condition   = "(df['FORECAST_REASON'] == '') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes') & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['IPES_SCORE'] > 0) & (df['RETIRED'] == 'No')"
# Parcel criteria for ADU development
ADU_Condition          = "(df['FORECAST_REASON'] == '') & (df['ADU_ALLOWED'] == 'Yes') & (df['Residential_Units'] == 1) & (df['OWNERSHIP_TYPE'] == 'Private')"

In [185]:
# get lookup lists
res_assigned_lookup = "Lookup_Lists/forecast_residential_assigned_units.csv"
res_zoned_lookup    = "Lookup_Lists/forecast_residential_zoned_units.csv"
# get zoned units lookups as data frames
dfResZoned    = pd.read_csv(res_zoned_lookup)
dfResAssigned = pd.read_csv(res_assigned_lookup)

In [53]:
# join assigned lookup to parcels
sdfAssignedParcels = dfResAssigned.merge(sdfParcels, how='left', on='APN')
sdfAssignedParcels['UnitPoolType'] = 'General'
sdfAssignedParcels.loc[sdfAssignedParcels['Source'] == 'Residential Bonus Unit', 'UnitPoolType'] = 'Bonus Unit'
# If the first 3 characters of the Source are 'ADU', set the UnitPoolType to 'ADU'
sdfAssignedParcels
sdfAssignedParcels.loc[sdfAssignedParcels['Source'].str.startswith('ADU'), 'UnitPoolType'] = 'ADU'
dfResAssignedParcelTotal = sdfAssignedParcels.groupby(['UnitPoolType', 'JURISDICTION'])['Unit Change'].sum().reset_index()
# These are more than we have in dfResZoned. Should they come out of TRPA?


In [186]:
# yearly development rate = future units / 28 years
dfResZoned['Yearly_Development_Rate'] = dfResZoned['Future_Units'] / 28
# Add Known Residential Allocations from 2023-2024
# set residential units to assigned total 
sdfParcels['FORECASTED_RESIDENTIAL_UNITS']        = sdfParcels.APN.map(dict(zip(dfResAssigned.APN, dfResAssigned['Unit Change'])))
# set forecast reason to assigned for all parcels in the assigned list
sdfParcels.loc[sdfParcels['APN'].isin(dfResAssigned.APN), 'FORECAST_REASON'] = 'Assigned'

In [ ]:
# Remove Assigned Units from Zoned Units
# Specific project are in chat

In [ ]:
# Deal with years anything assigned is 2035
# Does not matter about jurisdiction
# bring back random sort - should maybe be static? Date time?
# Assign CTC asset lands based on lookup list. Just a isin

In [187]:
# forecast CSLT Bonus Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'CSLT')&(dfResZoned.Unit_Pool == 'Bonus Unit'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, CSLT_Bonus_condition, target_sum, 'CSLT Bonus Units Built')
# # Forecast CSLT General Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'CSLT')&(dfResZoned.Unit_Pool == 'General'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, CSLT_General_Condition, target_sum, 'CSLT General Units Built')
# # Forecast EL General Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'EL')&(dfResZoned.Unit_Pool == 'General'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, EL_General_Condition, target_sum, 'EL General Units Built')
# # Forecast PL Bonus Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'PL')&(dfResZoned.Unit_Pool == 'Bonus Unit'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, PL_Bonus_Condition, target_sum, 'PL Bonus Units Built')
# # Forecast PL General Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'PL')&(dfResZoned.Unit_Pool == 'General'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, PL_General_Condition, target_sum, 'PL General Units Built')
# # Forecast WA Bonus Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'WA')&(dfResZoned.Unit_Pool == 'Bonus Unit'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, WA_Bonus_Condition, target_sum, 'WA Bonus Units Built')
# # Forecast WA General Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'WA')&(dfResZoned.Unit_Pool == 'General'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, WA_General_Condition, target_sum, 'WA General Units Built')
# # Forecast DG Bonus Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'DG')&(dfResZoned.Unit_Pool == 'Bonus Unit'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, DG_Bonus_Condition, target_sum, 'DG Bonus Units Built')
# # Forecast DG General Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'DG')&(dfResZoned.Unit_Pool == 'General'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, DG_General_Condition, target_sum, 'DG General Units Built')
# # Forecast TRPA Bonus Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'TRPA')&(dfResZoned.Unit_Pool == 'Bonus Unit'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, TRPA_Bonus_Condition, target_sum, 'TRPA Bonus Units Built')
# # Forecast TRPA General Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'TRPA')&(dfResZoned.Unit_Pool == 'General'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, TRPA_General_Condition, target_sum, 'TRPA General Units Built')
# forecast ADU Units
target_sum = dfResZoned.loc[(dfResZoned.Jurisdiction == 'TRPA')&(dfResZoned.Unit_Pool == 'ADU'), 'Future_Units'].values[0]
sdfParcels = forecast_residential_units(sdfParcels, ADU_Condition, target_sum, 'ADU Units Built')

In [ ]:
TotalDevelopment = dfResZoned.Future_Units.sum()
Development_2035 = (TotalDevelopment*.46).astype(int)
sdfParcels['Development_Year']=None
#Assining all development that's currently in the works as being cone by 2035
sdfParcels.loc[sdfParcels['FORECAST_REASON'] == 'Assigned', 'Development_Year'] = 2035
RemainingDevelopment_2035 = Development_2035 - sdfParcels.loc[sdfParcels['FORECAST_REASON'] == 'Assigned', 'FORECASTED_RESIDENTIAL_UNITS'].sum()

In [ ]:
Development_2035_Condition = sdfParcels['FORECASTED_RESIDENTIAL_UNITS'].where(sdfParcels['FORECAST_REASON'] != 'Assigned').cumsum()
sdfParcels.loc[Development_2035_Condition < RemainingDevelopment_2035, 'Development_Year'] = 2035
sdfParcels.loc[(sdfParcels['FORECAST_REASON']!= '') & (sdfParcels['Development_Year'].isnull()), 'Development_Year'] = 2050

In [189]:
grouped_df = sdfParcels.groupby('FORECAST_REASON').agg({'FORECASTED_RESIDENTIAL_UNITS':'sum'})

In [ ]:
# export to pickle
sdfParcels.to_pickle(parcel_pickle_part2)
# to feature class
sdfParcels.spatial.to_featureclass(Path(gdb)/'Parcel_Forecast')

### Tourist Accommodation Forecast

In [192]:
# lookup lists
tau_assigned_lookup = "Lookup_Lists/forecast_tourist_assigned_units.csv"
tau_zoned_lookup    = "Lookup_Lists/forecast_tourist_zoned_units.csv"
# get zoned units lookups as data frames
dfTouristZoned    = pd.read_csv(tau_zoned_lookup)
dfTouristAssigned = pd.read_csv(tau_assigned_lookup)

In [ ]:
sdfParcels['FORECASTED_TAU_UNITS'] = 0
# set tourist units to assigned total
sdfParcels['FORECASTED_TAU_UNITS'] = sdfParcels.APN.map(dict(zip(dfTouristAssigned.APN, dfTouristAssigned['Unit Change'])))
# set forecast reason to assigned for all parcels in the assigned list

### Commercial Floor Area Forecast

In [ ]:
# lookup lists
cfa_assigned_lookup = "Lookup_Lists/forecast_tourist_assigned_units.csv"
cfa_zoned_lookup    = "Lookup_Lists/forecast_tourist_zoned_units.csv"
# get zoned units lookups as data frames
dfCommercialZoned    = pd.read_csv(cfa_zoned_lookup)
dfCommercialAssigned = pd.read_csv(cfa_assigned_lookup)

In [ ]:
# condition = commercial allowed use, in town center, existing land use is vacant or commercial, existing coverage is less than 30% of parcel sqft, and not retired, and private ownership
comercial_condition = "(df['FORECAST_REASON'] == '') & (df['COUNTY_LANDUSE_DESCRIPTION'] == 'Commercial') & (df['EXISTING_LANDUSE'].isin(['Vacant','Commercial'])) & (df['CommercialFloorArea_SqFt'] < 0.3*df['PARCEL_SQFT']) & (df['OWNERSHIP_TYPE'] == 'Private') & (df['RETIRED'] == 'No')"